In [1]:
import ta
import sqlalchemy
import pandas as pd

In [2]:
engine = sqlalchemy.create_engine("sqlite:///usa.db")

In [3]:
names = pd.read_sql('select name from sqlite_master where type = "table"', engine)

In [4]:
names = names.name

In [5]:
framelist = [
    pd.read_sql(f"select Date, Close from {name}", engine)
    for name in names
]

In [6]:
def macd_decision(df):
    df["macd_diff"] = ta.trend.macd_diff(df.Close)
    df["Decision MACD"] = "nil"
    df.loc[(df.macd_diff > 0) & (df.macd_diff.shift(1) < 0), 'Decision MACD'] = 'BUY'

In [7]:
def rsi_decision(df):
    df["RSI"] = ta.momentum.rsi(df.Close, window=10)
    df["SMA200"] = ta.trend.sma_indicator(df.Close, window=200)
    df["Decision RSI/SMA"] = "nil"
    df.loc[(df.Close > df.SMA200) * (df.RSI < 30), "Decision RSI/SMA"] = "BUY"

In [8]:
for frame in framelist:
    macd_decision(frame)
    rsi_decision(frame)

In [9]:
for name, frame in zip(names, framelist):
    if frame['Decision MACD'].iloc[-1] == "BUY":
        print(f"Buy signal for MACD for {name}")
    if frame['Decision RSI/SMA'].iloc[-1] == "BUY":
        print(f"Buy signal for RSI/SMA for {name}")